In [21]:
# Import thư viện
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from datetime import datetime
import warnings
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, VotingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [22]:
# Load data
stock_data = pd.read_csv('./data/XRPUSDT20182024new.csv')
# stock_data = stock_data[:-100]

# Tiền xử lý
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
stock_data.drop(columns=['Change %'], errors='ignore', inplace=True)
stock_data = stock_data.sort_values('Date').reset_index(drop=True)

In [23]:
stock_data

,Date,Price,Open,High,Low,Vol
0,2018-05-26,0.6088,0.6056,0.6247,0.5989,99110000.0
1,2018-05-27,0.6046,0.6088,0.6119,0.5988,95150000.0
2,2018-05-28,0.5476,0.6047,0.6075,0.5460,158340000.0
3,2018-05-29,0.5994,0.5476,0.6088,0.5444,155950000.0
4,2018-05-30,0.5938,0.5994,0.6255,0.5786,132830000.0
...,...,...,...,...,...,...
2185,2024-05-20,0.5377,0.5096,0.5379,0.5067,672450000.0
2186,2024-05-21,0.5372,0.5377,0.5565,0.5321,812410000.0
2187,2024-05-22,0.5268,0.5372,0.5382,0.5236,509460000.0
2188,2024-05-23,0.5288,0.5269,0.5456,0.5091,948100000.0


In [24]:
def create_advanced_features(data):
    """
    Tạo các features kỹ thuật cho dự đoán giá cổ phiếu
    """
    feature_data = data.copy()
    
    # ==================== PRICE-BASED FEATURES ====================
    # Returns = % thay đổi giá
    # Đo lợi suất ngắn hạn
    # Thể hiện giá tăng hay giảm và mạnh hay yếu
    feature_data['Return_1d'] = feature_data['Price'].pct_change(1) * 100
    feature_data['Return_3d'] = feature_data['Price'].pct_change(3) * 100
    feature_data['Return_7d'] = feature_data['Price'].pct_change(7) * 100
    
    # Price ranges
    # HL_Range, HL_Range_Pct: độ giao động trong ngày, phản ánh sự giằng co mua bán -> Range lớn -> thị trường biến động mạnh dùng để đo risk/volatility
    feature_data['HL_Range'] = feature_data['High'] - feature_data['Low']
    feature_data['HL_Range_Pct'] = (feature_data['HL_Range'] / feature_data['Low']) * 100
    # OC_Range, OC_Range_Pct: giá bị đẩy mạnh hay bị bán mạnh trong ngày, giúp model nhận biết được ngày breakout/ panic
    feature_data['OC_Range'] = abs(feature_data['Open'] - feature_data['Price'])
    feature_data['OC_Range_Pct'] = (feature_data['OC_Range'] / feature_data['Open']) * 100
    
    # Price position: giá đóng cửa nằm ở đâu trong biên độ ngày: gần HIGH -> phe mua thắng , gần LOW -> phe bán thắng: Cô đọng tâm lý thị trường vào 1 số từ 0 -> 1 
    # Panic là cố gắng bán bằng mọi giá sống sót khi mà giá chứng khoáng giảm
    # Short Sell mượn cổ phiếu sau đó  bán rồi mua lại và trả lại cho công ty
    feature_data['Price_Position'] = (feature_data['Price'] - feature_data['Low']) / feature_data['HL_Range']
    
    # ==================== MOVING AVERAGES ====================
    # SMA: Trung bình giá k ngày : đại diện cho xu hướng ngắn/trung/dài hạn: Gía thường hồi về trung bình -> ML học được mối quan hệ: giá đang quá cao hay quá thấp hơn so với trend
    for period in [5, 7, 10, 14, 20, 30]:
        feature_data[f'SMA_{period}'] = feature_data['Price'].rolling(window=period).mean()
        feature_data[f'Price_to_SMA_{period}'] = (feature_data['Price'] / feature_data[f'SMA_{period}'] - 1) * 100
    
    # EMA phản ứng nhanh hơn SMA: thị trường tài chính thay đổi nhanh EMA giúp model bắt trend sớm
    for period in [5, 10, 20]:
        feature_data[f'EMA_{period}'] = feature_data['Price'].ewm(span=period, adjust=False).mean()
        feature_data[f'Price_to_EMA_{period}'] = (feature_data['Price'] / feature_data[f'EMA_{period}'] - 1) * 100
    
    # ==================== BOLLINGER BANDS ====================
    for period in [10, 20]:
        bb_middle = feature_data['Price'].rolling(window=period).mean()
        bb_std = feature_data['Price'].rolling(window=period).std()
        
        # biên trên/ dưới của giá bình thường
        feature_data[f'BB_Upper_{period}'] = bb_middle + (2 * bb_std)
        feature_data[f'BB_Lower_{period}'] = bb_middle - (2 * bb_std)
        # độ biến động của thị trường -> width lướn -> thị trường bất ổn
        feature_data[f'BB_Width_{period}'] = feature_data[f'BB_Upper_{period}'] - feature_data[f'BB_Lower_{period}']
        # giá đang ở đâu trong band: kết hợp giá + volatility
        feature_data[f'BB_Position_{period}'] = (feature_data['Price'] - feature_data[f'BB_Lower_{period}']) / feature_data[f'BB_Width_{period}']
    
    # ==================== MOMENTUM INDICATORS ====================
    # RSI: đó sức mạnh tăng/ giảm RSI >70: quá mua <30: quá bán rút gọn hành vi giá phức tạp thành 1 chỉ số
    for period in [7, 14]:
        price_delta = feature_data['Price'].diff()
        gain = (price_delta.where(price_delta > 0, 0)).rolling(window=period).mean()
        loss = (-price_delta.where(price_delta < 0, 0)).rolling(window=period).mean()
        rs_ratio = gain / loss
        feature_data[f'RSI_{period}'] = 100 - (100 / (1 + rs_ratio))
    
    # Stochastic : gái hiện tại so với min/max quá khứ -> giá đang ở đỉnh hay đáy cục bộ 
    for period in [14]:
        low_min = feature_data['Low'].rolling(window=period).min()
        high_max = feature_data['High'].rolling(window=period).max()
        feature_data[f'Stoch_{period}'] = 100 * (feature_data['Price'] - low_min) / (high_max - low_min)
    
    # ROC: tốc độ thay đổi giá: Moementum mạnh -> khả năng tiếp diễn xu hướng
    for period in [5, 10]:
        feature_data[f'ROC_{period}'] = ((feature_data['Price'] - feature_data['Price'].shift(period)) / feature_data['Price'].shift(period)) * 100
    
    # ==================== MACD ====================
    # So sánh xu hướng ngắn hạn và dài hạn
    ema_12 = feature_data['Price'].ewm(span=12, adjust=False).mean()
    ema_26 = feature_data['Price'].ewm(span=26, adjust=False).mean()
    feature_data['MACD'] = ema_12 - ema_26
    feature_data['MACD_Signal'] = feature_data['MACD'].ewm(span=9, adjust=False).mean()
    feature_data['MACD_Histogram'] = feature_data['MACD'] - feature_data['MACD_Signal']
    
    # ==================== VOLATILITY ====================
    for period in [5, 10, 20]:
        # Độ rủi ro ngắn hạn
        feature_data[f'Volatility_{period}'] = feature_data['Return_1d'].rolling(window=period).std()
        # Dao động trung bình thực tế
        # giá tăng 1% trong thị trường yên tĩnh khác với trong thị trường hỗn loạn
        feature_data[f'ATR_{period}'] = feature_data['HL_Range'].rolling(window=period).mean()
    
    # ==================== VOLUME FEATURES ====================
    # dòng tiền đột biến
    feature_data['Vol_Change'] = feature_data['Vol'].pct_change() * 100
    
    for period in [5, 10, 20]:
        feature_data[f'Vol_SMA_{period}'] = feature_data['Vol'].rolling(window=period).mean()
        # có tiền lớn vào hay không
        feature_data[f'Vol_Ratio_{period}'] = feature_data['Vol'] / feature_data[f'Vol_SMA_{period}']
    # giá tăng có được ủng hổ bởi volume không
    feature_data['Vol_Price_Corr_20'] = feature_data['Vol'].rolling(window=20).corr(feature_data['Price'])
    
    # ==================== LAG FEATURES ====================
    # TRÍ NHỚ QUÁ KHỨ
    for lag in [1, 2, 3, 5, 7]:
        # thị trường có tính phụ thuộc thời gian
        feature_data[f'Price_Lag_{lag}'] = feature_data['Price'].shift(lag)
        feature_data[f'Return_Lag_{lag}'] = feature_data['Return_1d'].shift(lag)
        feature_data[f'Vol_Lag_{lag}'] = feature_data['Vol'].shift(lag)
    
    # ==================== STATISTICAL FEATURES ====================
    # Mô tả phân phối giá trong một cửa sổ thời gian
    for period in [7, 14, 30]:
        feature_data[f'Price_Mean_{period}'] = feature_data['Price'].rolling(window=period).mean()
        feature_data[f'Price_Std_{period}'] = feature_data['Price'].rolling(window=period).std()
        feature_data[f'Price_Min_{period}'] = feature_data['Price'].rolling(window=period).min()
        feature_data[f'Price_Max_{period}'] = feature_data['Price'].rolling(window=period).max()
        feature_data[f'Price_Range_{period}'] = feature_data[f'Price_Max_{period}'] - feature_data[f'Price_Min_{period}']
    
    # ==================== TIME FEATURES ====================
    feature_data['Day_of_Week'] = feature_data['Date'].dt.dayofweek
    feature_data['Day_of_Month'] = feature_data['Date'].dt.day
    feature_data['Month'] = feature_data['Date'].dt.month
    feature_data['Quarter'] = feature_data['Date'].dt.quarter
    feature_data['Year'] = feature_data['Date'].dt.year
    
    # ==================== TARGET VARIABLE ====================
    # nhãn
    feature_data['Target_Price'] = feature_data['Price'].shift(-1)
    feature_data['Target_Return'] = ((feature_data['Target_Price'] - feature_data['Price']) / feature_data['Price']) * 100
    
    return feature_data

# Tạo features
feature_engineered_data = create_advanced_features(stock_data)

In [25]:
feature_engineered_data.drop(columns=['Target_Return'], errors='ignore', inplace=True)
feature_engineered_data.head()

,Date,Price,Open,High,Low,Vol,Return_1d,Return_3d,Return_7d,HL_Range,HL_Range_Pct,OC_Range,OC_Range_Pct,Price_Position,SMA_5,Price_to_SMA_5,SMA_7,Price_to_SMA_7,SMA_10,Price_to_SMA_10,SMA_14,Price_to_SMA_14,SMA_20,Price_to_SMA_20,SMA_30,Price_to_SMA_30,EMA_5,Price_to_EMA_5,EMA_10,Price_to_EMA_10,EMA_20,Price_to_EMA_20,BB_Upper_10,BB_Lower_10,BB_Width_10,BB_Position_10,BB_Upper_20,BB_Lower_20,BB_Width_20,BB_Position_20,RSI_7,RSI_14,Stoch_14,ROC_5,ROC_10,MACD,MACD_Signal,MACD_Histogram,Volatility_5,ATR_5,Volatility_10,ATR_10,Volatility_20,ATR_20,Vol_Change,Vol_SMA_5,Vol_Ratio_5,Vol_SMA_10,Vol_Ratio_10,Vol_SMA_20,Vol_Ratio_20,Vol_Price_Corr_20,Price_Lag_1,Return_Lag_1,Vol_Lag_1,Price_Lag_2,Return_Lag_2,Vol_Lag_2,Price_Lag_3,Return_Lag_3,Vol_Lag_3,Price_Lag_5,Return_Lag_5,Vol_Lag_5,Price_Lag_7,Return_Lag_7,Vol_Lag_7,Price_Mean_7,Price_Std_7,Price_Min_7,Price_Max_7,Price_Range_7,Price_Mean_14,Price_Std_14,Price_Min_14,Price_Max_14,Price_Range_14,Price_Mean_30,Price_Std_30,Price_Min_30,Price_Max_30,Price_Range_30,Day_of_Week,Day_of_Month,Month,Quarter,Year,Target_Price
0,2018-05-26,0.6088,0.6056,0.6247,0.5989,99110000.0,NaN,NaN,NaN,0.0258,4.307898,0.0032,0.528402,0.383721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.608800,0.000000,0.608800,0.000000,0.608800,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,26,5,2,2018,0.6046
1,2018-05-27,0.6046,0.6088,0.6119,0.5988,95150000.0,-0.689882,NaN,NaN,0.0131,2.187709,0.0042,0.689882,0.442748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.607400,-0.460981,0.608036,-0.565158,0.608400,-0.624589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000335,-0.000067,-0.000268,NaN,NaN,NaN,NaN,NaN,NaN,-3.995560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6088,NaN,99110000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,27,5,2,2018,0.5476
2,2018-05-28,0.5476,0.6047,0.6075,0.5460,158340000.0,-9.427721,NaN,NaN,0.0615,11.263736,0.0571,9.442699,0.026016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.587467,-6.786201,0.597048,-8.282071,0.602610,-9.128552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.005141,-0.001082,-0.004059,NaN,NaN,NaN,NaN,NaN,NaN,66.410930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6046,-0.689882,95150000.0,0.6088,NaN,99110000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,28,5,2,2018,0.5994
3,2018-05-29,0.5994,0.5476,0.6088,0.5444,155950000.0,9.459459,-1.544021,NaN,0.0644,11.829537,0.0518,9.459459,0.854037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.591444,1.345106,0.597476,0.322091,0.602304,-0.482125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.004715,-0.001808,-0.002907,NaN,NaN,NaN,NaN,NaN,NaN,-1.509410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5476,-9.427721,158340000.0,0.6046,-0.689882,95150000.0,0.6088,NaN,99110000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,29,5,2,2018,0.5938
4,2018-05-30,0.5938,0.5994,0.6255,0.5786,132830000.0,-0.934268,-1.786305,NaN,0.0469,8.105773,0.0056,0.934268,0.324094,0.59084,0.500982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.592230,0.265162,0.596807,-0.503897,0.601494,-1.279142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.004775,-0.002402,-0.002373,NaN,0.04234,NaN,NaN,NaN,NaN,-14.825265,128276000.0,1.035502,NaN,NaN,NaN,NaN,NaN,0.5994,9.459459,155950000.0,0.5476,-9.427721,158340000.0,0.6046,-0.689882,95150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,30,5,2,2018,0.6110


In [26]:
# Kiểm tra feature_engineered_data
if 'feature_engineered_data' not in globals():
    raise ValueError("feature_engineered_data chưa được tạo. Chạy cell tạo features trước.")

# Loại bỏ NaN
cleaned_data = feature_engineered_data.dropna()

cleaned_data

,Date,Price,Open,High,Low,Vol,Return_1d,Return_3d,Return_7d,HL_Range,HL_Range_Pct,OC_Range,OC_Range_Pct,Price_Position,SMA_5,Price_to_SMA_5,SMA_7,Price_to_SMA_7,SMA_10,Price_to_SMA_10,SMA_14,Price_to_SMA_14,SMA_20,Price_to_SMA_20,SMA_30,Price_to_SMA_30,EMA_5,Price_to_EMA_5,EMA_10,Price_to_EMA_10,EMA_20,Price_to_EMA_20,BB_Upper_10,BB_Lower_10,BB_Width_10,BB_Position_10,BB_Upper_20,BB_Lower_20,BB_Width_20,BB_Position_20,RSI_7,RSI_14,Stoch_14,ROC_5,ROC_10,MACD,MACD_Signal,MACD_Histogram,Volatility_5,ATR_5,Volatility_10,ATR_10,Volatility_20,ATR_20,Vol_Change,Vol_SMA_5,Vol_Ratio_5,Vol_SMA_10,Vol_Ratio_10,Vol_SMA_20,Vol_Ratio_20,Vol_Price_Corr_20,Price_Lag_1,Return_Lag_1,Vol_Lag_1,Price_Lag_2,Return_Lag_2,Vol_Lag_2,Price_Lag_3,Return_Lag_3,Vol_Lag_3,Price_Lag_5,Return_Lag_5,Vol_Lag_5,Price_Lag_7,Return_Lag_7,Vol_Lag_7,Price_Mean_7,Price_Std_7,Price_Min_7,Price_Max_7,Price_Range_7,Price_Mean_14,Price_Std_14,Price_Min_14,Price_Max_14,Price_Range_14,Price_Mean_30,Price_Std_30,Price_Min_30,Price_Max_30,Price_Range_30,Day_of_Week,Day_of_Month,Month,Quarter,Year,Target_Price
29,2018-06-24,0.4745,0.4903,0.4953,0.4406,172040000.0,-3.123724,-11.158959,-9.705043,0.0547,12.414889,0.0158,3.222517,0.619744,0.50412,-5.875585,0.515429,-7.940687,0.51998,-8.746490,0.531693,-10.756747,0.569170,-16.632992,0.585220,-18.919381,0.498330,-4.782035,0.519858,-8.725098,0.549207,-13.602680,0.573960,0.466000,0.107961,0.078736,0.703983,0.434357,0.269625,0.148883,27.473498,29.208689,21.214018,-13.585868,-14.734951,-0.032873,-0.023747,-0.009126,4.210226,0.03636,3.596211,0.03169,4.302296,0.037760,106.134675,126848000.0,1.356269,120155000.0,1.431817,131117000.0,1.312111,-0.160459,0.4898,1.555049,83460000.0,0.4823,-9.698558,192510000.0,0.5341,-1.074273,85200000.0,0.5491,2.006316,112200000.0,0.5255,-1.407129,100520000.0,0.515429,0.031712,0.4745,0.5491,0.0746,0.531693,0.032605,0.4745,0.5993,0.1248,0.585220,0.062960,0.4745,0.6880,0.2135,6,24,6,2,2018,0.4808
30,2018-06-25,0.4808,0.4745,0.4914,0.4726,96460000.0,1.327713,-0.311010,-10.681776,0.0188,3.977994,0.0063,1.327713,0.436170,0.49230,-2.335974,0.507214,-5.207717,0.51473,-6.591805,0.523229,-8.108994,0.559415,-14.053073,0.580953,-17.239480,0.492487,-2.373036,0.512757,-6.232323,0.542692,-11.404613,0.572995,0.456465,0.116530,0.208829,0.689874,0.428956,0.260919,0.198699,23.055295,25.963489,25.314861,-10.946472,-9.844365,-0.034135,-0.025825,-0.008310,4.606898,0.03494,3.566417,0.03060,4.223568,0.036240,-43.931644,125934000.0,0.765957,118300000.0,0.815385,128198000.0,0.752430,-0.165285,0.4745,-3.123724,172040000.0,0.4898,1.555049,83460000.0,0.4823,-9.698558,192510000.0,0.5399,-1.675469,101030000.0,0.5383,2.435775,128410000.0,0.507214,0.032243,0.4745,0.5491,0.0746,0.523229,0.028872,0.4745,0.5582,0.0837,0.580953,0.065589,0.4745,0.6880,0.2135,0,25,6,2,2018,0.4530
31,2018-06-26,0.4530,0.4809,0.4819,0.4517,71510000.0,-5.782030,-7.513271,-17.501366,0.0302,6.685853,0.0279,5.801622,0.043046,0.47608,-4.847925,0.493486,-8.204030,0.50673,-10.603280,0.515714,-12.160665,0.548400,-17.396061,0.575900,-21.340511,0.479325,-5.492018,0.501892,-9.741503,0.534150,-15.192334,0.574962,0.438498,0.136464,0.106269,0.675531,0.421269,0.254262,0.124797,11.156023,27.444254,9.904153,-15.184422,-15.009381,-0.036952,-0.028050,-0.008902,4.795809,0.03740,3.851466,0.03176,4.313295,0.036500,-25.865644,123196000.0,0.580457,114334000.0,0.625448,125436000.0,0.570092,-0.049480,0.4808,1.327713,96460000.0,0.4745,-3.123724,172040000.0,0.4898,1.555049,83460000.0,0.5341,-1.074273,85200000.0,0.5491,2.006316,112200000.0,0.493486,0.031893,0.4530,0.5399,0.0869,0.515714,0.032528,0.4530,0.5565,0.1035,0.575900,0.069432,0.4530,0.6880,0.2350,1,26,6,2,2018,0.4719
32,2018-06-27,0.4719,0.4530,0.4760,0.4486,101670000.0,4.172185,-0.547945,-12.594925,0.0274,6.107891,0.0189,4.172185,0.850365,0.47400,-0.443038,0.483771,-2.453933,0.50137,-5.877895,0.511571,-7.754817,0.538110,-12.304176,0.573377,-17.698081,0.476850,-1.038006,0.496439,-4.942956,0.528221,-10.662439,0.571445,0.431295,0.1401

In [27]:
# Kiểm tra dữ liệu sau khi dropna
if len(cleaned_data) == 0:
    raise ValueError(f"cleaned_data rỗng sau dropna(). feature_engineered_data có {len(feature_engineered_data)} dòng, nhưng tất cả đều có NaN.")

# Chọn features (loại bỏ Date, Target columns)
exclude_cols = ['Date', 'Target_Price', 'Target_Return']
feature_columns = [col for col in cleaned_data.columns if col not in exclude_cols]

# Prepare features and target
features = cleaned_data[feature_columns]
target = cleaned_data['Target_Price']

# Kiểm tra features và target
if len(features) == 0:
    raise ValueError(f"features rỗng. cleaned_data có {len(cleaned_data)} dòng.")
if len(target) == 0:
    raise ValueError(f"target rỗng.")

# Time-based split (không shuffle để giữ tính thời gian)
split_idx = int(len(features) * 0.5)
if split_idx == 0:
    raise ValueError(f"split_idx = 0. features chỉ có {len(features)} dòng, cần ít nhất 2 dòng để split.")

train_features, test_features = features[:split_idx], features[split_idx:]
train_target, test_target = target[:split_idx], target[split_idx:]

# Kiểm tra train và test sets
if len(train_features) == 0:
    raise ValueError(f"train_features rỗng. split_idx={split_idx}, features có {len(features)} dòng.")
if len(test_features) == 0:
    raise ValueError(f"test_features rỗng. split_idx={split_idx}, features có {len(features)} dòng.")

# Get date ranges for train and test
train_dates = cleaned_data.loc[train_features.index, 'Date']
test_dates = cleaned_data.loc[test_features.index, 'Date']

# Scaling
feature_scaler = RobustScaler()
scaled_train_features = feature_scaler.fit_transform(train_features)
scaled_test_features = feature_scaler.transform(test_features)

# Convert back to DataFrame
scaled_train_features = pd.DataFrame(scaled_train_features, columns=feature_columns, index=train_features.index)
scaled_test_features = pd.DataFrame(scaled_test_features, columns=feature_columns, index=test_features.index)

In [28]:
scaled_train_features

,Price,Open,High,Low,Vol,Return_1d,Return_3d,Return_7d,HL_Range,HL_Range_Pct,OC_Range,OC_Range_Pct,Price_Position,SMA_5,Price_to_SMA_5,SMA_7,Price_to_SMA_7,SMA_10,Price_to_SMA_10,SMA_14,Price_to_SMA_14,SMA_20,Price_to_SMA_20,SMA_30,Price_to_SMA_30,EMA_5,Price_to_EMA_5,EMA_10,Price_to_EMA_10,EMA_20,Price_to_EMA_20,BB_Upper_10,BB_Lower_10,BB_Width_10,BB_Position_10,BB_Upper_20,BB_Lower_20,BB_Width_20,BB_Position_20,RSI_7,RSI_14,Stoch_14,ROC_5,ROC_10,MACD,MACD_Signal,MACD_Histogram,Volatility_5,ATR_5,Volatility_10,ATR_10,Volatility_20,ATR_20,Vol_Change,Vol_SMA_5,Vol_Ratio_5,Vol_SMA_10,Vol_Ratio_10,Vol_SMA_20,Vol_Ratio_20,Vol_Price_Corr_20,Price_Lag_1,Return_Lag_1,Vol_Lag_1,Price_Lag_2,Return_Lag_2,Vol_Lag_2,Price_Lag_3,Return_Lag_3,Vol_Lag_3,Price_Lag_5,Return_Lag_5,Vol_Lag_5,Price_Lag_7,Return_Lag_7,Vol_Lag_7,Price_Mean_7,Price_Std_7,Price_Min_7,Price_Max_7,Price_Range_7,Price_Mean_14,Price_Std_14,Price_Min_14,Price_Max_14,Price_Range_14,Price_Mean_30,Price_Std_30,Price_Min_30,Price_Max_30,Price_Range_30,Day_of_Week,Day_of_Month,Month,Quarter,Year
29,0.919614,1.004153,0.926903,0.860472,-0.603180,-0.724459,-1.408662,-0.730059,1.602484,1.069611,0.649635,0.284294,0.262557,1.081135,-1.276762,1.139853,-1.352865,1.173515,-1.155225,1.244482,-1.141830,1.469454,-1.261698,1.593297,-1.092827,1.055761,-1.304257,1.205146,-1.375673,1.359825,-1.254967,1.111418,1.328782,0.662929,-0.844329,1.645642,1.550689,1.625353,-0.609728,-0.610964,-0.734874,-0.554949,-1.292699,-0.873449,-1.754497,-1.264511,-1.538144,0.174408,0.808429,-0.147907,0.610413,0.098475,0.916324,2.466440,-0.602640,0.921235,-0.670308,0.904354,-0.765601,0.588620,-0.572822,1.001608,0.351480,-0.607603,0.961415,-2.264370,-0.601947,1.239014,-0.254033,-0.607436,1.319400,0.464679,-0.605870,1.192926,-0.328917,-0.606173,1.139853,1.200956,1.226824,1.118961,0.910107,1.244482,0.688740,1.378715,1.225894,0.993113,1.593297,1.071999,2.470073,1.403019,0.944412,0.75,0.533333,-0.142857,-0.333333,-1.0
30,0.953376,0.919491,0.907209,1.049263,-0.607101,0.303014,-0.003226,-0.806669,0.115942,-0.250043,-0.043796,-0.227214,-0.176650,1.017243,-0.483609,1.095400,-0.878441,1.144685,-0.863269,1.197080,-0.846810,1.413804,-1.053815,1.568328,-0.987178,1.023937,-0.624234,1.164864,-0.963449,1.322401,-1.034307,1.107068,1.263253,0.763913,-0.569471,1.582323,1.503823,1.551533,-0.507892,-0.737086,-0.864658,-0.454989,-1.035620,-0.566792,-1.823125,-1.377215,-1.404164,0.287358,0.747978,-0.157905,0.560630,0.071434,0.842698,-0.965060,-0.602688,-0.361706,-0.670413,-0.143478,-0.765777,-0.200085,-0.580293,0.919614,-0.728982,-0.603009,1.001608,0.359027,-0.607603,0.961415,-2.265464,-0.601871,1.270096,-0.394445,-0.606449,1.261522,0.559933,-0.604728,1.095400,1.231539,1.226824,1.118961,0.910107,1.197080,0.533925,1.378715,1.039837,0.426997,1.568328,1.145017,2.470073,1.403019,0.944412,-0.75,0.600000,-0.142857,-0.333333,-1.0
31,0.804394,0.953784,0.859235,0.925959,-0.608395,-1.338044,-0.936335,-1.341563,0.587992,0.173506,1.532847,0.980530,-1.117211,0.929568,-1.046485,1.021106,-1.398579,1.100754,-1.406817,1.154998,-1.298259,1.350966,-1.323184,1.538754,-1.245092,0.952255,-1.504674,1.103236,-1.543754,1.273332,-1.414549,1.115929,1.139766,0.998839,-0.786158,1.517949,1.437133,1.495094,-0.658965,-1.076764,-0.805438,-0.830634,-1.448400,-0.890657,-1.976346,-1.497943,-1.501353,0.341150,0.852703,-0.062255,0.613610,0.102253,0.855292,-0.551953,-0.602831,-0.764858,-0.670637,-0.466337,-0.765944,-0.457037,-0.401021,0.953376,0.298982,-0.606928,0.919614,-0.731671,-0.603009,1.001608,0.359200,-0.607526,1.239014,-0.254159,-0.607269,1.319400,0.460600,-0.605568,1.021106,1.211398,1.085470,1.074290,1.159980,1.154998,0.685551,1.230131,1.032141,0.699725,1.538754,1.251740,2.218978,1.403019,1.110982,-0.50,0.666667,-0.142857,-0.333333,-1.0
32,0.905681,0.804287,0.829441,0.907670,-0.606830,0.959570,-0.033923,-0.956727,0.472050,0.083104,0.875912,0.540659,0.814324,0.918324,-0.059441,0.968535,-0.400404,1.071320,-0.766535,1.131797,-0.807347,1.292264,-0.912894,1.523987,-1.0160

In [29]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
from sklearn.svm import SVR
from sklearn.linear_model import Ridge

X_train_scaled = scaled_train_features.dropna()
X_test_scaled = scaled_test_features.dropna()
y_train = train_target
X_train = train_features
X_test = test_features
rf = RandomForestRegressor(
    n_estimators=500,
    max_depth=8,
    min_samples_leaf=20,
    random_state=42,
    n_jobs=-1
)

# linear_layer1 = Ridge(alpha=1.0)
# linear_layer1.fit(X_train_scaled, train_target)

rf.fit(X_train_scaled, train_target)


# SVR
scaler_svr = StandardScaler()
X_train_svr = scaler_svr.fit_transform(X_train_scaled)
X_test_svr  = scaler_svr.transform(X_test_scaled)

svr_model = SVR(kernel='rbf', C=100, gamma='scale')
svr_model.fit(X_train_svr, y_train)

cleaned_data.loc[X_train.index, 'RF_Pred_Tomorrow'] = rf.predict(X_train_scaled)
cleaned_data.loc[X_test.index,  'RF_Pred_Tomorrow'] = rf.predict(X_test_scaled)

# cleaned_data.loc[X_train.index, 'RF_Pred_Tomorrow_linear'] = linear_layer1.predict(X_train_scaled)
# cleaned_data.loc[X_test.index,  'RF_Pred_Tomorrow_linear'] = linear_layer1.predict(X_test_scaled)

cleaned_data.loc[X_train.index, 'SVR_Pred_Tomorrow'] = svr_model.predict(X_train_svr)
cleaned_data.loc[X_test.index,  'SVR_Pred_Tomorrow'] = svr_model.predict(X_test_svr)

# RF dự đoán Close[t+1] tại ngày t
# → dùng cho ngày t+1
cleaned_data['RF_Pred_Today'] = cleaned_data['RF_Pred_Tomorrow'].shift(1)
# cleaned_data['RF_Pred_Today_linear'] = cleaned_data['RF_Pred_Tomorrow_linear'].shift(1)
cleaned_data['SVR_Pred_Today'] = cleaned_data['SVR_Pred_Tomorrow'].shift(1)
# cleaned_data.dropna()
cleaned_data


,Date,Price,Open,High,Low,Vol,Return_1d,Return_3d,Return_7d,HL_Range,HL_Range_Pct,OC_Range,OC_Range_Pct,Price_Position,SMA_5,Price_to_SMA_5,SMA_7,Price_to_SMA_7,SMA_10,Price_to_SMA_10,SMA_14,Price_to_SMA_14,SMA_20,Price_to_SMA_20,SMA_30,Price_to_SMA_30,EMA_5,Price_to_EMA_5,EMA_10,Price_to_EMA_10,EMA_20,Price_to_EMA_20,BB_Upper_10,BB_Lower_10,BB_Width_10,BB_Position_10,BB_Upper_20,BB_Lower_20,BB_Width_20,BB_Position_20,RSI_7,RSI_14,Stoch_14,ROC_5,ROC_10,MACD,MACD_Signal,MACD_Histogram,Volatility_5,ATR_5,Volatility_10,ATR_10,Volatility_20,ATR_20,Vol_Change,Vol_SMA_5,Vol_Ratio_5,Vol_SMA_10,Vol_Ratio_10,Vol_SMA_20,Vol_Ratio_20,Vol_Price_Corr_20,Price_Lag_1,Return_Lag_1,Vol_Lag_1,Price_Lag_2,Return_Lag_2,Vol_Lag_2,Price_Lag_3,Return_Lag_3,Vol_Lag_3,Price_Lag_5,Return_Lag_5,Vol_Lag_5,Price_Lag_7,Return_Lag_7,Vol_Lag_7,Price_Mean_7,Price_Std_7,Price_Min_7,Price_Max_7,Price_Range_7,Price_Mean_14,Price_Std_14,Price_Min_14,Price_Max_14,Price_Range_14,Price_Mean_30,Price_Std_30,Price_Min_30,Price_Max_30,Price_Range_30,Day_of_Week,Day_of_Month,Month,Quarter,Year,Target_Price,RF_Pred_Tomorrow,SVR_Pred_Tomorrow,RF_Pred_Today,SVR_Pred_Today
29,2018-06-24,0.4745,0.4903,0.4953,0.4406,172040000.0,-3.123724,-11.158959,-9.705043,0.0547,12.414889,0.0158,3.222517,0.619744,0.50412,-5.875585,0.515429,-7.940687,0.51998,-8.746490,0.531693,-10.756747,0.569170,-16.632992,0.585220,-18.919381,0.498330,-4.782035,0.519858,-8.725098,0.549207,-13.602680,0.573960,0.466000,0.107961,0.078736,0.703983,0.434357,0.269625,0.148883,27.473498,29.208689,21.214018,-13.585868,-14.734951,-0.032873,-0.023747,-0.009126,4.210226,0.03636,3.596211,0.03169,4.302296,0.037760,106.134675,126848000.0,1.356269,120155000.0,1.431817,131117000.0,1.312111,-0.160459,0.4898,1.555049,83460000.0,0.4823,-9.698558,192510000.0,0.5341,-1.074273,85200000.0,0.5491,2.006316,112200000.0,0.5255,-1.407129,100520000.0,0.515429,0.031712,0.4745,0.5491,0.0746,0.531693,0.032605,0.4745,0.5993,0.1248,0.585220,0.062960,0.4745,0.6880,0.2135,6,24,6,2,2018,0.4808,0.475647,0.489642,NaN,NaN
30,2018-06-25,0.4808,0.4745,0.4914,0.4726,96460000.0,1.327713,-0.311010,-10.681776,0.0188,3.977994,0.0063,1.327713,0.436170,0.49230,-2.335974,0.507214,-5.207717,0.51473,-6.591805,0.523229,-8.108994,0.559415,-14.053073,0.580953,-17.239480,0.492487,-2.373036,0.512757,-6.232323,0.542692,-11.404613,0.572995,0.456465,0.116530,0.208829,0.689874,0.428956,0.260919,0.198699,23.055295,25.963489,25.314861,-10.946472,-9.844365,-0.034135,-0.025825,-0.008310,4.606898,0.03494,3.566417,0.03060,4.223568,0.036240,-43.931644,125934000.0,0.765957,118300000.0,0.815385,128198000.0,0.752430,-0.165285,0.4745,-3.123724,172040000.0,0.4898,1.555049,83460000.0,0.4823,-9.698558,192510000.0,0.5399,-1.675469,101030000.0,0.5383,2.435775,128410000.0,0.507214,0.032243,0.4745,0.5491,0.0746,0.523229,0.028872,0.4745,0.5582,0.0837,0.580953,0.065589,0.4745,0.6880,0.2135,0,25,6,2,2018,0.4530,0.473935,0.503779,0.475647,0.489642
31,2018-06-26,0.4530,0.4809,0.4819,0.4517,71510000.0,-5.782030,-7.513271,-17.501366,0.0302,6.685853,0.0279,5.801622,0.043046,0.47608,-4.847925,0.493486,-8.204030,0.50673,-10.603280,0.515714,-12.160665,0.548400,-17.396061,0.575900,-21.340511,0.479325,-5.492018,0.501892,-9.741503,0.534150,-15.192334,0.574962,0.438498,0.136464,0.106269,0.675531,0.421269,0.254262,0.124797,11.156023,27.444254,9.904153,-15.184422,-15.009381,-0.036952,-0.028050,-0.008902,4.795809,0.03740,3.851466,0.03176,4.313295,0.036500,-25.865644,123196000.0,0.580457,114334000.0,0.625448,125436000.0,0.570092,-0.049480,0.4808,1.327713,96460000.0,0.4745,-3.123724,172040000.0,0.4898,1.555049,83460000.0,0.5341,-1.074273,85200000.0,0.5491,2.006316,112200000.0,0.493486,0.031893,0.4530,0.5399,0.0869,0.515714,0.032528,0.4530,0.5565,0.1035,0.575900,0.069432,0.4530,0.6880,0.2350,1,26,6,2,2018,0.4719,0.460765,0.474085,0.473935,0.503779
32,2018-06-27,0.4719,0.4530,0.4760,0.4486,101670000.0,4.172185,-0.547945,-12.594925,0.0274,6.107891,0.0189,4.172185,0.850365,0.47400,-0.443038,0.483771,-2.4539

In [30]:
cleaned_data.dropna()

,Date,Price,Open,High,Low,Vol,Return_1d,Return_3d,Return_7d,HL_Range,HL_Range_Pct,OC_Range,OC_Range_Pct,Price_Position,SMA_5,Price_to_SMA_5,SMA_7,Price_to_SMA_7,SMA_10,Price_to_SMA_10,SMA_14,Price_to_SMA_14,SMA_20,Price_to_SMA_20,SMA_30,Price_to_SMA_30,EMA_5,Price_to_EMA_5,EMA_10,Price_to_EMA_10,EMA_20,Price_to_EMA_20,BB_Upper_10,BB_Lower_10,BB_Width_10,BB_Position_10,BB_Upper_20,BB_Lower_20,BB_Width_20,BB_Position_20,RSI_7,RSI_14,Stoch_14,ROC_5,ROC_10,MACD,MACD_Signal,MACD_Histogram,Volatility_5,ATR_5,Volatility_10,ATR_10,Volatility_20,ATR_20,Vol_Change,Vol_SMA_5,Vol_Ratio_5,Vol_SMA_10,Vol_Ratio_10,Vol_SMA_20,Vol_Ratio_20,Vol_Price_Corr_20,Price_Lag_1,Return_Lag_1,Vol_Lag_1,Price_Lag_2,Return_Lag_2,Vol_Lag_2,Price_Lag_3,Return_Lag_3,Vol_Lag_3,Price_Lag_5,Return_Lag_5,Vol_Lag_5,Price_Lag_7,Return_Lag_7,Vol_Lag_7,Price_Mean_7,Price_Std_7,Price_Min_7,Price_Max_7,Price_Range_7,Price_Mean_14,Price_Std_14,Price_Min_14,Price_Max_14,Price_Range_14,Price_Mean_30,Price_Std_30,Price_Min_30,Price_Max_30,Price_Range_30,Day_of_Week,Day_of_Month,Month,Quarter,Year,Target_Price,RF_Pred_Tomorrow,SVR_Pred_Tomorrow,RF_Pred_Today,SVR_Pred_Today
30,2018-06-25,0.4808,0.4745,0.4914,0.4726,96460000.0,1.327713,-0.311010,-10.681776,0.0188,3.977994,0.0063,1.327713,0.436170,0.49230,-2.335974,0.507214,-5.207717,0.51473,-6.591805,0.523229,-8.108994,0.559415,-14.053073,0.580953,-17.239480,0.492487,-2.373036,0.512757,-6.232323,0.542692,-11.404613,0.572995,0.456465,0.116530,0.208829,0.689874,0.428956,0.260919,0.198699,23.055295,25.963489,25.314861,-10.946472,-9.844365,-0.034135,-0.025825,-0.008310,4.606898,0.03494,3.566417,0.03060,4.223568,0.036240,-43.931644,125934000.0,0.765957,118300000.0,0.815385,128198000.0,0.752430,-0.165285,0.4745,-3.123724,172040000.0,0.4898,1.555049,83460000.0,0.4823,-9.698558,192510000.0,0.5399,-1.675469,101030000.0,0.5383,2.435775,128410000.0,0.507214,0.032243,0.4745,0.5491,0.0746,0.523229,0.028872,0.4745,0.5582,0.0837,0.580953,0.065589,0.4745,0.6880,0.2135,0,25,6,2,2018,0.4530,0.473935,0.503779,0.475647,0.489642
31,2018-06-26,0.4530,0.4809,0.4819,0.4517,71510000.0,-5.782030,-7.513271,-17.501366,0.0302,6.685853,0.0279,5.801622,0.043046,0.47608,-4.847925,0.493486,-8.204030,0.50673,-10.603280,0.515714,-12.160665,0.548400,-17.396061,0.575900,-21.340511,0.479325,-5.492018,0.501892,-9.741503,0.534150,-15.192334,0.574962,0.438498,0.136464,0.106269,0.675531,0.421269,0.254262,0.124797,11.156023,27.444254,9.904153,-15.184422,-15.009381,-0.036952,-0.028050,-0.008902,4.795809,0.03740,3.851466,0.03176,4.313295,0.036500,-25.865644,123196000.0,0.580457,114334000.0,0.625448,125436000.0,0.570092,-0.049480,0.4808,1.327713,96460000.0,0.4745,-3.123724,172040000.0,0.4898,1.555049,83460000.0,0.5341,-1.074273,85200000.0,0.5491,2.006316,112200000.0,0.493486,0.031893,0.4530,0.5399,0.0869,0.515714,0.032528,0.4530,0.5565,0.1035,0.575900,0.069432,0.4530,0.6880,0.2350,1,26,6,2,2018,0.4719,0.460765,0.474085,0.473935,0.503779
32,2018-06-27,0.4719,0.4530,0.4760,0.4486,101670000.0,4.172185,-0.547945,-12.594925,0.0274,6.107891,0.0189,4.172185,0.850365,0.47400,-0.443038,0.483771,-2.453933,0.50137,-5.877895,0.511571,-7.754817,0.538110,-12.304176,0.573377,-17.698081,0.476850,-1.038006,0.496439,-4.942956,0.528221,-10.662439,0.571445,0.431295,0.140150,0.289725,0.653993,0.422227,0.231766,0.214324,24.512744,37.042002,25.000000,-2.156334,-10.199810,-0.037231,-0.029886,-0.007345,4.002891,0.03018,4.256279,0.03262,4.489796,0.036865,42.175919,105028000.0,0.968028,114449000.0,0.888343,124994000.0,0.813399,0.025943,0.4530,-5.782030,71510000.0,0.4808,1.327713,96460000.0,0.4745,-3.123724,172040000.0,0.4823,-9.698558,192510000.0,0.5399,-1.675469,101030000.0,0.483771,0.025014,0.4530,0.5341,0.0811,0.511571,0.034231,0.4530,0.5565,0.1035,0.573377,0.071830,0.4530,0.6880,0.2350,2,27,6,2,2018,0.4479,0.463477,0.495977,0.460765,0.474085
33,2018-06-28,0.4479,0.4716,0.4743,0.4456,141220000.0,-5.085823,-6.842762,-16.139300,0.0287,6.440754,0.0237,5.025445,0.080139,0.46562,-3.805678,0.471457,-

In [31]:
cleaned_data

,Date,Price,Open,High,Low,Vol,Return_1d,Return_3d,Return_7d,HL_Range,HL_Range_Pct,OC_Range,OC_Range_Pct,Price_Position,SMA_5,Price_to_SMA_5,SMA_7,Price_to_SMA_7,SMA_10,Price_to_SMA_10,SMA_14,Price_to_SMA_14,SMA_20,Price_to_SMA_20,SMA_30,Price_to_SMA_30,EMA_5,Price_to_EMA_5,EMA_10,Price_to_EMA_10,EMA_20,Price_to_EMA_20,BB_Upper_10,BB_Lower_10,BB_Width_10,BB_Position_10,BB_Upper_20,BB_Lower_20,BB_Width_20,BB_Position_20,RSI_7,RSI_14,Stoch_14,ROC_5,ROC_10,MACD,MACD_Signal,MACD_Histogram,Volatility_5,ATR_5,Volatility_10,ATR_10,Volatility_20,ATR_20,Vol_Change,Vol_SMA_5,Vol_Ratio_5,Vol_SMA_10,Vol_Ratio_10,Vol_SMA_20,Vol_Ratio_20,Vol_Price_Corr_20,Price_Lag_1,Return_Lag_1,Vol_Lag_1,Price_Lag_2,Return_Lag_2,Vol_Lag_2,Price_Lag_3,Return_Lag_3,Vol_Lag_3,Price_Lag_5,Return_Lag_5,Vol_Lag_5,Price_Lag_7,Return_Lag_7,Vol_Lag_7,Price_Mean_7,Price_Std_7,Price_Min_7,Price_Max_7,Price_Range_7,Price_Mean_14,Price_Std_14,Price_Min_14,Price_Max_14,Price_Range_14,Price_Mean_30,Price_Std_30,Price_Min_30,Price_Max_30,Price_Range_30,Day_of_Week,Day_of_Month,Month,Quarter,Year,Target_Price,RF_Pred_Tomorrow,SVR_Pred_Tomorrow,RF_Pred_Today,SVR_Pred_Today
29,2018-06-24,0.4745,0.4903,0.4953,0.4406,172040000.0,-3.123724,-11.158959,-9.705043,0.0547,12.414889,0.0158,3.222517,0.619744,0.50412,-5.875585,0.515429,-7.940687,0.51998,-8.746490,0.531693,-10.756747,0.569170,-16.632992,0.585220,-18.919381,0.498330,-4.782035,0.519858,-8.725098,0.549207,-13.602680,0.573960,0.466000,0.107961,0.078736,0.703983,0.434357,0.269625,0.148883,27.473498,29.208689,21.214018,-13.585868,-14.734951,-0.032873,-0.023747,-0.009126,4.210226,0.03636,3.596211,0.03169,4.302296,0.037760,106.134675,126848000.0,1.356269,120155000.0,1.431817,131117000.0,1.312111,-0.160459,0.4898,1.555049,83460000.0,0.4823,-9.698558,192510000.0,0.5341,-1.074273,85200000.0,0.5491,2.006316,112200000.0,0.5255,-1.407129,100520000.0,0.515429,0.031712,0.4745,0.5491,0.0746,0.531693,0.032605,0.4745,0.5993,0.1248,0.585220,0.062960,0.4745,0.6880,0.2135,6,24,6,2,2018,0.4808,0.475647,0.489642,NaN,NaN
30,2018-06-25,0.4808,0.4745,0.4914,0.4726,96460000.0,1.327713,-0.311010,-10.681776,0.0188,3.977994,0.0063,1.327713,0.436170,0.49230,-2.335974,0.507214,-5.207717,0.51473,-6.591805,0.523229,-8.108994,0.559415,-14.053073,0.580953,-17.239480,0.492487,-2.373036,0.512757,-6.232323,0.542692,-11.404613,0.572995,0.456465,0.116530,0.208829,0.689874,0.428956,0.260919,0.198699,23.055295,25.963489,25.314861,-10.946472,-9.844365,-0.034135,-0.025825,-0.008310,4.606898,0.03494,3.566417,0.03060,4.223568,0.036240,-43.931644,125934000.0,0.765957,118300000.0,0.815385,128198000.0,0.752430,-0.165285,0.4745,-3.123724,172040000.0,0.4898,1.555049,83460000.0,0.4823,-9.698558,192510000.0,0.5399,-1.675469,101030000.0,0.5383,2.435775,128410000.0,0.507214,0.032243,0.4745,0.5491,0.0746,0.523229,0.028872,0.4745,0.5582,0.0837,0.580953,0.065589,0.4745,0.6880,0.2135,0,25,6,2,2018,0.4530,0.473935,0.503779,0.475647,0.489642
31,2018-06-26,0.4530,0.4809,0.4819,0.4517,71510000.0,-5.782030,-7.513271,-17.501366,0.0302,6.685853,0.0279,5.801622,0.043046,0.47608,-4.847925,0.493486,-8.204030,0.50673,-10.603280,0.515714,-12.160665,0.548400,-17.396061,0.575900,-21.340511,0.479325,-5.492018,0.501892,-9.741503,0.534150,-15.192334,0.574962,0.438498,0.136464,0.106269,0.675531,0.421269,0.254262,0.124797,11.156023,27.444254,9.904153,-15.184422,-15.009381,-0.036952,-0.028050,-0.008902,4.795809,0.03740,3.851466,0.03176,4.313295,0.036500,-25.865644,123196000.0,0.580457,114334000.0,0.625448,125436000.0,0.570092,-0.049480,0.4808,1.327713,96460000.0,0.4745,-3.123724,172040000.0,0.4898,1.555049,83460000.0,0.5341,-1.074273,85200000.0,0.5491,2.006316,112200000.0,0.493486,0.031893,0.4530,0.5399,0.0869,0.515714,0.032528,0.4530,0.5565,0.1035,0.575900,0.069432,0.4530,0.6880,0.2350,1,26,6,2,2018,0.4719,0.460765,0.474085,0.473935,0.503779
32,2018-06-27,0.4719,0.4530,0.4760,0.4486,101670000.0,4.172185,-0.547945,-12.594925,0.0274,6.107891,0.0189,4.172185,0.850365,0.47400,-0.443038,0.483771,-2.4539

In [ ]:
layer2_features = [
    'Open',
    'High',
    'Low',
    'Vol',
    'RF_Pred_Today',
    'SVR_Pred_Today'
]

cleaned_data['Target_Close_Today'] = cleaned_data['Price']
df_l2 = cleaned_data.dropna(subset=layer2_features + ['Target_Close_Today'])

X2 = df_l2[layer2_features]
y2 = df_l2['Target_Close_Today']

split_idx = int(len(X2) * 0.4)

X2_train, X2_test = X2[:split_idx], X2[split_idx:]
y2_train, y2_test = y2[:split_idx], y2[split_idx:]


from sklearn.linear_model import Ridge

l2_model = Ridge(alpha=1.0)
l2_model.fit(X2_train, y2_train)

y2_pred = l2_model.predict(X2_test)

from sklearn.metrics import mean_absolute_error

mae_l2 = mean_absolute_error(y2_test, y2_pred)
direction_acc = (
    np.sign(y2_test.diff().dropna()) ==
    np.sign(pd.Series(y2_pred, index=y2_test.index).diff().dropna())
).mean() * 100

print("\n" + "="*80)
print("LAYER 2 PERFORMANCE")
print("="*80)
print(f"MAE: {mae_l2:.6f}")
print(f"Direction Accuracy: {direction_acc:.2f}%")




LAYER 2 PERFORMANCE
MAE: 0.026920
Direction Accuracy: 57.07%


In [33]:
cleaned_data

,Date,Price,Open,High,Low,Vol,Return_1d,Return_3d,Return_7d,HL_Range,HL_Range_Pct,OC_Range,OC_Range_Pct,Price_Position,SMA_5,Price_to_SMA_5,SMA_7,Price_to_SMA_7,SMA_10,Price_to_SMA_10,SMA_14,Price_to_SMA_14,SMA_20,Price_to_SMA_20,SMA_30,Price_to_SMA_30,EMA_5,Price_to_EMA_5,EMA_10,Price_to_EMA_10,EMA_20,Price_to_EMA_20,BB_Upper_10,BB_Lower_10,BB_Width_10,BB_Position_10,BB_Upper_20,BB_Lower_20,BB_Width_20,BB_Position_20,RSI_7,RSI_14,Stoch_14,ROC_5,ROC_10,MACD,MACD_Signal,MACD_Histogram,Volatility_5,ATR_5,Volatility_10,ATR_10,Volatility_20,ATR_20,Vol_Change,Vol_SMA_5,Vol_Ratio_5,Vol_SMA_10,Vol_Ratio_10,Vol_SMA_20,Vol_Ratio_20,Vol_Price_Corr_20,Price_Lag_1,Return_Lag_1,Vol_Lag_1,Price_Lag_2,Return_Lag_2,Vol_Lag_2,Price_Lag_3,Return_Lag_3,Vol_Lag_3,Price_Lag_5,Return_Lag_5,Vol_Lag_5,Price_Lag_7,Return_Lag_7,Vol_Lag_7,Price_Mean_7,Price_Std_7,Price_Min_7,Price_Max_7,Price_Range_7,Price_Mean_14,Price_Std_14,Price_Min_14,Price_Max_14,Price_Range_14,Price_Mean_30,Price_Std_30,Price_Min_30,Price_Max_30,Price_Range_30,Day_of_Week,Day_of_Month,Month,Quarter,Year,Target_Price,RF_Pred_Tomorrow,SVR_Pred_Tomorrow,RF_Pred_Today,SVR_Pred_Today,Target_Close_Today
29,2018-06-24,0.4745,0.4903,0.4953,0.4406,172040000.0,-3.123724,-11.158959,-9.705043,0.0547,12.414889,0.0158,3.222517,0.619744,0.50412,-5.875585,0.515429,-7.940687,0.51998,-8.746490,0.531693,-10.756747,0.569170,-16.632992,0.585220,-18.919381,0.498330,-4.782035,0.519858,-8.725098,0.549207,-13.602680,0.573960,0.466000,0.107961,0.078736,0.703983,0.434357,0.269625,0.148883,27.473498,29.208689,21.214018,-13.585868,-14.734951,-0.032873,-0.023747,-0.009126,4.210226,0.03636,3.596211,0.03169,4.302296,0.037760,106.134675,126848000.0,1.356269,120155000.0,1.431817,131117000.0,1.312111,-0.160459,0.4898,1.555049,83460000.0,0.4823,-9.698558,192510000.0,0.5341,-1.074273,85200000.0,0.5491,2.006316,112200000.0,0.5255,-1.407129,100520000.0,0.515429,0.031712,0.4745,0.5491,0.0746,0.531693,0.032605,0.4745,0.5993,0.1248,0.585220,0.062960,0.4745,0.6880,0.2135,6,24,6,2,2018,0.4808,0.475647,0.489642,NaN,NaN,0.4745
30,2018-06-25,0.4808,0.4745,0.4914,0.4726,96460000.0,1.327713,-0.311010,-10.681776,0.0188,3.977994,0.0063,1.327713,0.436170,0.49230,-2.335974,0.507214,-5.207717,0.51473,-6.591805,0.523229,-8.108994,0.559415,-14.053073,0.580953,-17.239480,0.492487,-2.373036,0.512757,-6.232323,0.542692,-11.404613,0.572995,0.456465,0.116530,0.208829,0.689874,0.428956,0.260919,0.198699,23.055295,25.963489,25.314861,-10.946472,-9.844365,-0.034135,-0.025825,-0.008310,4.606898,0.03494,3.566417,0.03060,4.223568,0.036240,-43.931644,125934000.0,0.765957,118300000.0,0.815385,128198000.0,0.752430,-0.165285,0.4745,-3.123724,172040000.0,0.4898,1.555049,83460000.0,0.4823,-9.698558,192510000.0,0.5399,-1.675469,101030000.0,0.5383,2.435775,128410000.0,0.507214,0.032243,0.4745,0.5491,0.0746,0.523229,0.028872,0.4745,0.5582,0.0837,0.580953,0.065589,0.4745,0.6880,0.2135,0,25,6,2,2018,0.4530,0.473935,0.503779,0.475647,0.489642,0.4808
31,2018-06-26,0.4530,0.4809,0.4819,0.4517,71510000.0,-5.782030,-7.513271,-17.501366,0.0302,6.685853,0.0279,5.801622,0.043046,0.47608,-4.847925,0.493486,-8.204030,0.50673,-10.603280,0.515714,-12.160665,0.548400,-17.396061,0.575900,-21.340511,0.479325,-5.492018,0.501892,-9.741503,0.534150,-15.192334,0.574962,0.438498,0.136464,0.106269,0.675531,0.421269,0.254262,0.124797,11.156023,27.444254,9.904153,-15.184422,-15.009381,-0.036952,-0.028050,-0.008902,4.795809,0.03740,3.851466,0.03176,4.313295,0.036500,-25.865644,123196000.0,0.580457,114334000.0,0.625448,125436000.0,0.570092,-0.049480,0.4808,1.327713,96460000.0,0.4745,-3.123724,172040000.0,0.4898,1.555049,83460000.0,0.5341,-1.074273,85200000.0,0.5491,2.006316,112200000.0,0.493486,0.031893,0.4530,0.5399,0.0869,0.515714,0.032528,0.4530,0.5565,0.1035,0.575900,0.069432,0.4530,0.6880,0.2350,1,26,6,2,2018,0.4719,0.460765,0.474085,0.473935,0.503779,0.4530
32,2018-06-27,0.4719,0.4530,0.4760,0.4486,101670000.0,4.172185,-0.547945,-12.594925,0.0274,6.107891,0.0189,4.172185,0.8